# Prueba de la extración de feature vectors

In [19]:
#Importación de librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne import io
import random
import scipy.signal as signal
from datetime import datetime, timedelta
from FVfunctions import getMeData, pot4signals, BANDAS
from features_stats import stats_features
from dataset_reader import get_seizure_events, get_seizure_array

#### 1. Lectura

In [6]:
DATA_DIR = r"../eeg_dataset/physionet.org/files/siena-scalp-eeg/1.0.0/"
path01 = rf"{DATA_DIR}PN05/PN05-2.edf"
raw01 = io.read_raw_edf(path01)
paciente = "PN05"
realizacion = "2"

Extracting EDF parameters from C:\Users\gonza\Documents\ITBA\2024 1C\PSIB\TP FINAL PSIB\eeg_dataset\physionet.org\files\siena-scalp-eeg\1.0.0\PN05\PN05-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\gonza\AppData\Local\Temp\ipykernel_19040\2785477139.py:3: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw01 = io.read_raw_edf(path01)
C:\Users\gonza\AppData\Local\Temp\ipykernel_19040\2785477139.py:3: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw01 = io.read_raw_edf(path01)
C:\Users\gonza\AppData\Local\Temp\ipykernel_19040\2785477139.py:3: RuntimeWarning: Highpass cutoff frequency 15.91549 is greater than lowpass cutoff frequency 15.0, setting values to 0 and Nyquist.
  raw01 = io.read_raw_edf(path01)


#### 2. Info del archivo

In [7]:
info = raw01.info
print(info)
print(info['ch_names'])

<Info | 8 non-empty values
 bads: []
 ch_names: EEG Fp1, EEG F3, EEG C3, EEG P3, EEG O1, EEG F7, EEG T3, EEG T5, ...
 chs: 37 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: 2016-01-01 06:46:02 UTC
 nchan: 37
 projs: []
 sfreq: 512.0 Hz
 subject_info: 1 item (dict)
>
['EEG Fp1', 'EEG F3', 'EEG C3', 'EEG P3', 'EEG O1', 'EEG F7', 'EEG T3', 'EEG T5', 'EEG Fc1', 'EEG Fc5', 'EEG Cp1', 'EEG Cp5', 'EEG F9', 'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG F4', 'EEG C4', 'EEG P4', 'EEG O2', 'EEG F8', 'EEG T4', 'EEG T6', 'EEG Fc2', 'EEG Fc6', 'EEG Cp2', 'EEG Cp6', 'EEG F10', 'EKG EKG', 'SPO2', 'HR', '1', '2', 'EEG FP2', 'EEG P9', 'EEG P10', 'MK']


#### 3. Obtención de las mediciones a utilizar y otros datos importantes

In [9]:
# Se obtiene el nombre de todos los canales
ch_nms = info["ch_names"]

# Se obtienen los canales seleccionados del lazo izquierdo
filt_ch_nms = ['EEG T3','EEG T5','EEG F7','EEG F3','EEG C3','EEG P3']

# Seleccionar los datos de los canales filtrados
raw01_filt = raw01.pick(filt_ch_nms)

# Obtener los datos de los canales filtrados por nombre
data_namefilt = raw01_filt.get_data()

# Se convierten las mediciones a microvoltios
data_namefilt = data_namefilt * 1e6

# Dimensiones de 'data_filt'
dim_data_filt = np.shape(data_namefilt)

# Verifición
print("Canales filtrados:", filt_ch_nms)
print(f"Cantidad de canales resultantes: {dim_data_filt[0]}")
print(f"Cantidad de datos en cada canal: {dim_data_filt[1]}")

Canales filtrados: ['EEG T3', 'EEG T5', 'EEG F7', 'EEG F3', 'EEG C3', 'EEG P3']
Cantidad de canales resultantes: 6
Cantidad de datos en cada canal: 4733440


In [12]:
# Frecuencia de muestreo y cantidad de muestras tomadas
fs = info["sfreq"]
len_data = dim_data_filt[1]

# Array de instantes
start = 0
stop = (1 / fs) * len_data
arr_t = np.arange(start=start, stop=stop, step=(1 / fs))

# Verificación de datos
print("Frecuencia de muestreo:", fs)
print("Instantes [s]:", arr_t[0], "...", arr_t[-1])

Frecuencia de muestreo: 512.0
Instantes [s]: 0.0 ... 9244.998046875


#### 4. getMeData

In [22]:
# Encontramos las seizures del archivo
path_seiz = rf"{DATA_DIR}PN05/Seizures-list-PN05.txt"
seizures = get_seizure_events(path_seiz)
seizures_in_02 = [seiz for seiz in seizures if seiz["file_name"] == "PN05-2.edf"]
arr_mtx_t_epi = get_seizure_array(seizures_in_02)
seizures_in_02

[{'seizure_number': 2,
  'file_name': 'PN05-2.edf',
  'registration_start_time': '06.46.02',
  'registration_end_time': '09.19.47',
  'seizure_start_time': '08.45.25',
  'seizure_end_time': '08.46.00'}]

In [23]:
# Matriz de inicio de registro y final de registro
seiz = seizures_in_02[0] # una sola !!
mtx_t_reg = np.array([seiz['registration_start_time'], seiz['registration_end_time']])

# Utilización de la función para una única señal
arr_seg, arr_labels = getMeData(sig=data_namefilt[0], mtx_t_reg=mtx_t_reg, arr_mtx_t_epi=arr_mtx_t_epi)
arr_seg

array([[ -6.44861197, -14.41007862, -21.80983335, ...,  -4.48831135,
         -4.21901134,  -2.8510888 ],
       [ -5.88095295,  -3.27032485,  -3.43263646, ...,  12.59162582,
          7.36264985,   1.6076962 ],
       [ -0.83301453,  -1.25101932,  -2.04141079, ..., -23.45783771,
        -24.17600637, -25.67897428],
       ...,
       [  0.27892381,   2.93108426,   5.18785783, ...,  -3.71822747,
          0.20019193,   2.69750499],
       [  0.43146587,  -2.62196953,  -3.05619533, ...,  -5.29018175,
         -3.0133207 ,   0.32962988],
       [  0.82660546,   2.6589613 ,   2.31354896, ...,  -0.96321824,
          0.98874327,   2.69161266]])

In [24]:
# Potencia
pot_seg = pot4signals(arr_seg, fs, divisor=1)
pot_names = [f"potAbs{band.capitalize()}" for band in BANDAS.keys()]

# Estadistica
stats_data = stats_features(arr_seg)
stats_names = list(stats_data.values())[:-1]
stats_seg = stats_data["matriz de features stat"]

# Feature fector
arr_fv = np.hstack((pot_seg, stats_seg))
columnas = pot_names + stats_names
df_fv = pd.DataFrame(data=arr_fv, columns=columnas)
df_fv

,potAbsDelta,potAbsTheta,potAbsAlpha,potAbsBeta,potAbsGamma,kurtosis,RMS,skewness,media,desvio estandar
0,217.644236,49.269005,89.263621,95.580341,55.646721,0.585607,21.551527,0.285625,6.390122,20.582387
1,178.283112,68.107614,68.265979,55.249263,21.184491,0.058750,20.155965,-0.071286,-1.154695,20.122863
2,148.265309,81.879942,29.160342,32.561074,17.223644,0.168532,20.059845,-0.216411,-4.901466,19.451813
3,131.956727,91.343940,18.468434,41.782452,16.610201,-0.005117,18.680859,0.482669,-0.225125,18.679502
4,171.847087,80.592865,21.834394,24.324345,9.639249,-0.669140,17.785635,-0.056150,1.242607,17.742174
5,49.951440,266.194512,118.488910,33.968660,10.059972,-0.137855,22.604525,-0.397360,2.536449,22.461767
6,60.522752,694.580842,171.390084,42.324166,8.917114,-0.446337,30.976213,-0.094013,-0.918058,30.962606
7,46.869211,908.317318,155.500322,71.472523,11.021553,-0.840184,33.378961,-0.038565,0.574652,33.374014
8,76.570210,566.605445,24.871353,73.751879,16.436683,-0.485127,26.430783,0.213435,2.710957,26.291386
9,80.895259,92.963097,126.074663,172.348406,22.912454,0.375545,24.919111,0.079699,-3.971633,24.600574
